In [2]:
import re
import requests
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
from tqdm import tqdm

# page nos. 1 to 574
telugu_url = 'https://dsal.uchicago.edu/cgi-bin/app/gwynn_query.py?page='

In [3]:
soups = {}

def get_soup(page):
    if page in soups:
        return soups[page]
    
    res = requests.get(telugu_url + str(page))
    soup = BeautifulSoup(res.text)
    soups[page] = soup
    return soup


In [4]:
for i in tqdm(range(1,575)):
    get_soup(i)

100%|██████████| 574/574 [04:50<00:00,  1.98it/s]


In [5]:
import pickle
pickle.dump(soups, open('./telugu_soups.pickle', 'wb'))

RecursionError: maximum recursion depth exceeded

In [10]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [43]:
def process_entry(div):
    return {
        'telugu': div.select_one('te').text.strip(),
        'transliteration': ''.join(s.strip() for s in div.select_one('hw').children if type(s) == NavigableString),
        'subentry_count': len([1 for b in div.select('b') if b.text.strip().isnumeric() or len(b.text.strip().split())>1]),
        'text': div.text.strip(),
        'html': str(div)
    }

sorted_entries = pd.concat([pd.DataFrame([process_entry(div) for div in soups[page].select('div.hw_result > div')]) for page in soups.keys()], ignore_index=True).sort_values('subentry_count', ascending=False)

In [44]:
sorted_entries[sorted_entries.subentry_count > 4].sample(10).sort_values('subentry_count')

,telugu,transliteration,subentry_count,text,html
15734,పరిభ్రమించు,paribhramiñcu,5,"పరిభ్రమించు paribhramiñcu paribhramincu v.i. 1 to revolve, circle round. 2 to rotate, gyrate. 3 to wander about, go from place to place. 4 waaDi mati paribhramincindi his mind became unstable.","<div> \n\t\t\t\n\t\t\t<hw><te>పరిభ్రమించు</te> paribhramiñcu</hw> <b>paribhramincu</b> <i>v.i</i>. <b>1</b> to revolve, circle round. <b>2</b> to rotate, gyrate. <b>3</b> to wander about, go from place to place. <b>4</b> <b>waaDi mati paribhramincindi</b> his mind became unstable. \n\t\t\t</div>"
26436,సాఫు,",sāphu, sāpu",5,"సాఫు, సాపు sāphu, sāpu saaphu, saapu I. n. (also ~ prati) fair copy of s.g written. II. adj. 1 smooth, not wrinkled. 2 straight, ~ gaa niDupugaa unnakarra a long straight stick. 3 level.","<div> \n\t\t\t\n\t\t\t<hw><te>సాఫు</te>, <te>సాపు</te> sāphu, sāpu</hw> <b>saaphu</b>, <b>saapu</b> I. <i>n</i>. (<i>also</i> <b>~ prati</b>) fair copy of s.g written. II. <i>adj</i>. <b>1</b> smooth, not wrinkled. <b>2</b> straight, <b>~ gaa niDupugaa unnakarra</b> a long straight stick. <b>3</b> level. \n\t\t\t</div>"
20008,మళ్లీ,",maḷlī, maḷlā",6,"మళ్లీ, మళ్లా maḷlī, maḷlā maLLii, maLLaa advbl. particle 1 again. 2 but still, and yet, however; tanaku raamaayaNam aNTee nijam katha ani nammakam leedanii; ~ bhaarataanni wiśwasistaananii ceppEEru he said he did not believe that the Ramayana was a true story, but still he believed in the Mahabharata; nannu tiTTEEDu, ~ neenaNTee aayanaki preema leedani kaadu he scolded me but that is not to say that he had no love towards me.","<div> \n\t\t\t\n\t\t\t<hw><te>మళ్లీ</te>, <te>మళ్లా</te> maḷlī, maḷlā</hw> <b>maLLii</b>, <b>maLLaa</b> <i>advbl. particle</i> <b>1</b> again. <b>2</b> but still, and yet, however; <b>tanaku raamaayaNam aNTee nijam katha ani nammakam leedanii</b>; <b>~ bhaarataanni wiśwasistaananii ceppEEru</b> he said he did not believe that the Ramayana was a true story, but still he believed in the Mahabharata; <b>nannu tiTTEEDu</b>, <b>~ neenaNTee aayanaki preema leedani kaadu</b> he scolded me but that is not to say that he had no love towards me. \n\t\t\t</div>"
1365,అప్పటికి,appaṭiki,6,"అప్పటికి appaṭiki appaTiki I. adv. 1 by that time. 2 dial. by evening time. II. advbl. n. suffixed to vbl. adj. 1 by the time that; miiru wacceeT ~ neenu akkaDee unnaanu by the time that you came, I was at that very place. 2 since ( = because); miiru uttaram raaseeT ~ aayana ceeyaka tappaleedu since you wrote him a letter, he had to do it without fail.","<div> \n\t\t\t\n\t\t\t<hw><te>అప్పటికి</te> appaṭiki</hw> <b>appaTiki</b> I. <i>adv</i>. <b>1</b> by that time. <b>2</b> <i>dial</i>. by evening time. II. <i>advbl. n. suffixed to vbl. adj</i>. <b>1</b> by the time that; <b>miiru wacceeT ~ neenu akkaDee unnaanu</b> by the time that you came, I was at that very place. <b>2</b> since ( = because); <b>miiru uttaram raaseeT ~ aayana ceeyaka tappaleedu</b> since you wrote him a letter, he had to do it without fail. \n\t\t\t</div>"
20181,మాడు,māḍu,6,"మాడు māḍu maaDu2 I. n. food burnt while cooking. II. v.i. 1 to be burnt, be scorched; see pramaaNam sense 2. 2 to become black due to scorching; also fig., e.g., waaDi mokham maaDindi his face grew black (with anger, jealousy or shame). 3 to starve; waaLLu annam leeka maaDi castunnaaru they are starving to death. 4 to be ruined; see aDugumaaDipoowu.","<div> \n\t\t\t\n\t\t\t<hw><te>మాడు</te> māḍu</hw> <b>maaDu</b><sup>2</sup> I. <i>n</i>. food burnt while cooking. II. <i>v.i</i>. <b>1</b> to be burnt, be scorched; <i>see</i> <b>pramaaNam</b> <i>sense 2</i>. <b>2</b> to become black due to scorching; <i>also fig</i>., e.g., <b>waaDi mokham maaDindi</b> his face grew black (with anger, jealousy or shame). <b>3</b> to starve; <b>waaLLu annam leeka maaDi castunnaaru</b> they are starving to death. <b>4</b> to be ruined; <i>see</i> <b>aDugumaaDipoowu</b>. \n\t\t\t</div>"
12086,తొలి,toli,6,"తొలి toli toli I. n. beginning. II. adj. 1 first. 2 old, ancient. 3 former, previo

In [39]:
sorted_entries.subentry_count.value_counts().sort_index(ascending=False).pipe(lambda x: x/x.sum()).cumsum()[-20:]

19    0.001807
18    0.002132
17    0.002349
16    0.002855
15    0.003397
14    0.004047
13    0.004625
12    0.005565
11    0.007408
10    0.009793
9     0.012322
8     0.015249
7     0.020886
6     0.029450
5     0.045566
4     0.074872
3     0.137421
2     0.274590
1     0.386283
0     1.000000
Name: subentry_count, dtype: float64

In [45]:
with open('telugu_entries.html', 'w') as outfile:

    for html in sorted_entries[sorted_entries.subentry_count >= 4].html:
        outfile.write(f'<div>{html}</div><br/>')
